In [ ]:
# 📦 Import library
import psycopg2
from dotenv import load_dotenv
import os
from urllib.parse import urlparse

# 📄 Load environment variables
load_dotenv()

# 🔗 Parsing DB_URI dari .env
db_uri = os.getenv("DB_URI")
print("DB_URI Loaded:", db_uri)

parsed = urlparse(db_uri)

# 🔌 Coba Connect ke Database PostgreSQL
try:
    conn = psycopg2.connect(
        dbname=parsed.path.lstrip("/"),
        user=parsed.username,
        password=parsed.password,
        host=parsed.hostname,
        port=parsed.port
    )
    print("✅ Berhasil terhubung ke database!")
    conn.close()

except Exception as e:
    print("❌ Gagal terhubung ke database!")
    print(str(e))


In [ ]:
# 📦 Import API KEY

import requests

# Ganti dengan API key kamu
api_key = os.getenv("GEMINI_API_TOKEN")

# Coba call endpoint Gemini simple
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={api_key}"

headers = {
    "Content-Type": "application/json",
}

body = {
    "contents": [
        {
            "parts": [
                {"text": "Halo, bisa dengar saya?"}
            ]
        }
    ]
}

# Kirim request
response = requests.post(url, headers=headers, json=body)

print("Status Code:", response.status_code)
print("Response Body:", response.text)


In [ ]:
# Import
from src.tag.src.text2sqlchain import generate_sql
from src.tag.src.query_executor import execute_text2sql_response
from src.tag.src.answer_generator import generate_answer
from src.tag.database.db_connection import connect_db
from src.tag.database.schema_loader import load_schema
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Connect ke database
conn = connect_db()

# Load schema database
schema = load_schema(conn)

# Pertanyaan awal
question = "Pasal mana saja yang merujuk ke Pasal 57 atau dirujuk oleh Pasal 57 dalam Undang-Undang Nomor 27 Tahun 2022, dan berapa totalnya?"

#  1. Generate Text2SQL Response
response = generate_sql(schema, question, top_k=100)

#  2. Execute SQL dari Response
rows, columns = execute_text2sql_response(conn, response)

#  3. Generate Jawaban Natural
jawaban = generate_answer(columns, rows, question)

# Print SQL
print("===== TEXT2SQL RESPONSE =====")
print(response)
# print("==============================")

#  Print jawaban
print("\n===== JAWABAN NATURAL =====")
print(jawaban)
